In [1]:
###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

###
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

###
import multiprocessing
import warnings
import gc
import datetime
import graphviz
from time import time
from tqdm import tqdm
from tqdm import tqdm_notebook

In [115]:
# load data
# train
# train = pd.read_csv('../../data/extracted/train.csv', index_col='TransactionID')
train = pd.read_csv('../../data/extracted/train_final(knn).csv', index_col=0)

id_df = pd.read_csv('../../data/extracted/d6_d7_d8_d9_d12-d13_d14_ids (2).csv', index_col=0)

In [80]:
id_df = id_df.set_index('id_01')

,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,...,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
id_01,,,,,,,,,,,,,,,,,,,,,
-8.0,214911.4,0.0,0.0,0.2,-3.4,6.0,-60.8,0.0,0.0,100.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
-8.0,214911.4,0.0,0.0,0.2,-3.4,6.0,-60.8,0.0,0.0,100.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
-8.0,214911.4,0.0,0.0,0.2,-3.4,6.0,-60.8,0.0,0.0,100.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
-8.0,214911.4,0.0,0.0,0.2,-3.4,6.0,-60.8,0.0,0.0,100.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
0.0,70787.0,1.2,0.0,0.4,-5.0,12.8,-46.2,0.0,0.0,100.0,...,1.0,7.0,124.0,32.0,164.0,3.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-18.0,253451.6,0.0,0.0,-2.2,-27.2,18.0,-59.4,0.0,0.0,100.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
-18.0,253451.6,0.0,0.0,-2.2,-27.2,18.0,-59.4,0.0,0.0,100.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
-18.0,253451.6,0.0,0.0,-2.2,-27.2,18.0,-59.4,0.0,0.0,100.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0


In [26]:
id_df = id_df.iloc[:,9:47]
id_df

,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
0,346.8,361.0,508.4,24.8,3.0,13.4,284.8,149.2,2.0,2.0,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
1,346.8,361.0,508.4,24.8,3.0,13.4,284.8,149.2,2.0,2.0,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
2,346.8,361.0,508.4,24.8,3.0,13.4,284.8,149.2,2.0,2.0,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
3,346.8,361.0,508.4,24.8,3.0,13.4,284.8,149.2,2.0,2.0,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
4,542.0,144.0,422.6,19.4,3.0,13.4,337.8,144.6,2.0,1.0,1.0,7.0,124.0,32.0,164.0,3.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,453.0,467.0,366.8,14.0,3.0,13.8,299.0,155.0,2.0,2.0,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590536,453.0,467.0,366.8,14.0,3.0,13.8,299.0,155.0,2.0,2.0,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590537,453.0,467.0,366.8,14.0,3.0,13.8,299.0,155.0,2.0,2.0,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590538,453.0,467.0,366.8,14.0,3.0,13.8,299.0,155.0,2.0,2.0,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0


In [66]:
id_df.isnull().sum()

id_01    0
id_02    0
id_03    0
id_04    0
id_05    0
id_06    0
id_07    0
id_08    0
id_09    0
id_10    0
id_11    0
id_12    0
id_13    0
id_14    0
id_15    0
id_16    0
id_17    0
id_18    0
id_19    0
id_20    0
id_21    0
id_22    0
id_23    0
id_24    0
id_25    0
id_26    0
id_27    0
id_28    0
id_29    0
id_30    0
id_31    0
id_32    0
id_33    0
id_34    0
id_35    0
id_36    0
id_37    0
id_38    0
dtype: int64

In [111]:
train = pd.merge(train ,id_df, how='left', left_index=True, right_index=True)

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000.0,0.0,86400.0,68.500000,4.0,13926.0,441.0,150.0,1.0,142.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001.0,0.0,86401.0,29.000000,4.0,2755.0,404.0,150.0,2.0,102.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002.0,0.0,86469.0,59.000000,4.0,4663.0,490.0,150.0,4.0,166.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003.0,0.0,86499.0,50.000000,4.0,18132.0,567.0,150.0,2.0,117.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004.0,0.0,86506.0,50.000000,1.0,4497.0,514.0,150.0,2.0,102.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577535.0,0.0,15811047.0,49.000000,4.0,6550.0,523.8,150.0,4.0,226.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3577536.0,0.0,15811049.0,39.500000,4.0,10444.0,225.0,150.0,2.0,224.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3577537.0,0.0,15811079.0,30.953125,4.0,12037.0,595.0,150.0,2.0,224.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
train.reset_index(inplace=True)
train.shape

(590540, 116)

In [92]:
id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Data columns (total 38 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id_01   590540 non-null  float64
 1   id_02   590540 non-null  float64
 2   id_03   590540 non-null  float64
 3   id_04   590540 non-null  float64
 4   id_05   590540 non-null  float64
 5   id_06   590540 non-null  float64
 6   id_07   590540 non-null  float64
 7   id_08   590540 non-null  float64
 8   id_09   590540 non-null  float64
 9   id_10   590540 non-null  float64
 10  id_11   590540 non-null  float64
 11  id_12   590540 non-null  float64
 12  id_13   590540 non-null  float64
 13  id_14   590540 non-null  float64
 14  id_15   590540 non-null  float64
 15  id_16   590540 non-null  float64
 16  id_17   590540 non-null  float64
 17  id_18   590540 non-null  float64
 18  id_19   590540 non-null  float64
 19  id_20   590540 non-null  float64
 20  id_21   590540 non-null  float64
 21  id_22   59

In [69]:
train.isnull().sum()

isFraud                0
TransactionDT          0
TransactionAmt         0
ProductCD              0
card1                  0
                   ...  
id_34             590540
id_35             590540
id_36             590540
id_37             590540
id_38             590540
Length: 114, dtype: int64

In [70]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000.0,0.0,86400.0,68.5,4.0,13926.0,441.0,150.0,1.0,142.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001.0,0.0,86401.0,29.0,4.0,2755.0,404.0,150.0,2.0,102.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002.0,0.0,86469.0,59.0,4.0,4663.0,490.0,150.0,4.0,166.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003.0,0.0,86499.0,50.0,4.0,18132.0,567.0,150.0,2.0,117.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004.0,0.0,86506.0,50.0,1.0,4497.0,514.0,150.0,2.0,102.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
id_df

,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,...,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
0,-8.0,214911.4,0.0,0.0,0.2,-3.4,6.0,-60.8,0.0,0.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
1,-8.0,214911.4,0.0,0.0,0.2,-3.4,6.0,-60.8,0.0,0.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
2,-8.0,214911.4,0.0,0.0,0.2,-3.4,6.0,-60.8,0.0,0.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
3,-8.0,214911.4,0.0,0.0,0.2,-3.4,6.0,-60.8,0.0,0.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
4,0.0,70787.0,1.2,0.0,0.4,-5.0,12.8,-46.2,0.0,0.0,...,1.0,7.0,124.0,32.0,164.0,3.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,-18.0,253451.6,0.0,0.0,-2.2,-27.2,18.0,-59.4,0.0,0.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590536,-18.0,253451.6,0.0,0.0,-2.2,-27.2,18.0,-59.4,0.0,0.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590537,-18.0,253451.6,0.0,0.0,-2.2,-27.2,18.0,-59.4,0.0,0.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590538,-18.0,253451.6,0.0,0.0,-2.2,-27.2,18.0,-59.4,0.0,0.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0


In [23]:
train.to_csv('../../data/extracted/train.csv')

In [97]:
train = pd.concat([train, id_df], axis=1)
train

,index,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,...,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
0,0,2987000.0,0.0,86400.0,68.500000,4.0,13926.0,441.0,150.0,1.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
1,1,2987001.0,0.0,86401.0,29.000000,4.0,2755.0,404.0,150.0,2.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
2,2,2987002.0,0.0,86469.0,59.000000,4.0,4663.0,490.0,150.0,4.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
3,3,2987003.0,0.0,86499.0,50.000000,4.0,18132.0,567.0,150.0,2.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
4,4,2987004.0,0.0,86506.0,50.000000,1.0,4497.0,514.0,150.0,2.0,...,1.0,7.0,124.0,32.0,164.0,3.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,590535,3577535.0,0.0,15811047.0,49.000000,4.0,6550.0,523.8,150.0,4.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590536,590536,3577536.0,0.0,15811049.0,39.500000,4.0,10444.0,225.0,150.0,2.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590537,590537,3577537.0,0.0,15811079.0,30.953125,4.0,12037.0,595.0,150.0,2.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590538,590538,3577538.0,0.0,15811088.0,117.000000,4.0,7826.0,481.0,150.0,2.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0


In [98]:
train.drop(['index'], axis=1, inplace=True)

In [102]:
train.to_csv('../../data/extracted/train_final(knn)1.csv')

In [116]:
train = pd.read_csv('../../data/extracted/train_final(knn).csv', index_col=0)

In [117]:
train

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_29.2,id_30.2,id_31.2,id_32.2,id_33.2,id_34.2,id_35.2,id_36.2,id_37.2,id_38.2
0,2987000.0,0.0,86400.0,68.500000,4.0,13926.0,441.0,150.0,1.0,142.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
1,2987001.0,0.0,86401.0,29.000000,4.0,2755.0,404.0,150.0,2.0,102.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
2,2987002.0,0.0,86469.0,59.000000,4.0,4663.0,490.0,150.0,4.0,166.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
3,2987003.0,0.0,86499.0,50.000000,4.0,18132.0,567.0,150.0,2.0,117.0,...,2.0,74.0,103.0,25.6,260.0,4.0,2.0,2.0,2.0,2.0
4,2987004.0,0.0,86506.0,50.000000,1.0,4497.0,514.0,150.0,2.0,102.0,...,1.0,7.0,124.0,32.0,164.0,3.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535.0,0.0,15811047.0,49.000000,4.0,6550.0,523.8,150.0,4.0,226.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590536,3577536.0,0.0,15811049.0,39.500000,4.0,10444.0,225.0,150.0,2.0,224.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590537,3577537.0,0.0,15811079.0,30.953125,4.0,12037.0,595.0,150.0,2.0,224.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0
590538,3577538.0,0.0,15811088.0,117.000000,4.0,7826.0,481.0,150.0,2.0,224.0,...,2.0,74.0,103.0,27.2,260.0,4.0,2.0,2.0,2.0,2.0


In [118]:
train.columns.to_list()

['TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'P_emaildomain',
 'C1',
 'C2',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D10',
 'D11',
 'D15',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'PCA_V1',
 'PCA_V2',
 'PCA_V3',
 'PCA_V4',
 'PCA_V5',
 'PCA_V6',
 'PCA_V7',
 'PCA_V8',
 'PCA_V9',
 'PCA_V10',
 'PCA_V11',
 'PCA_V12',
 'PCA_V13',
 'PCA_V14',
 'PCA_V15',
 'PCA_V16',
 'PCA_V17',
 'PCA_V18',
 'PCA_V19',
 'PCA_V20',
 'PCA_V21',
 'PCA_V22',
 'PCA_V23',
 'PCA_V24',
 'PCA_V25',
 'PCA_V26',
 'PCA_V27',
 'PCA_V28',
 'PCA_V29',
 'PCA_V30',
 'dist2',
 'C3',
 'id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_

In [123]:
train.drop(train.loc[:,'id_01.1':'id_38.1'], axis=1, inplace=True)

In [124]:
train.columns.tolist()

['TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'P_emaildomain',
 'C1',
 'C2',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D10',
 'D11',
 'D15',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'PCA_V1',
 'PCA_V2',
 'PCA_V3',
 'PCA_V4',
 'PCA_V5',
 'PCA_V6',
 'PCA_V7',
 'PCA_V8',
 'PCA_V9',
 'PCA_V10',
 'PCA_V11',
 'PCA_V12',
 'PCA_V13',
 'PCA_V14',
 'PCA_V15',
 'PCA_V16',
 'PCA_V17',
 'PCA_V18',
 'PCA_V19',
 'PCA_V20',
 'PCA_V21',
 'PCA_V22',
 'PCA_V23',
 'PCA_V24',
 'PCA_V25',
 'PCA_V26',
 'PCA_V27',
 'PCA_V28',
 'PCA_V29',
 'PCA_V30',
 'dist2',
 'C3',
 'id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_

In [125]:
train.to_csv('../../data/extracted/train_final(knn)1.csv')

In [126]:
train = pd.read_csv('../../data/extracted/train_final(knn).csv', index_col=0)

In [127]:
train.columns.tolist()

['TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'P_emaildomain',
 'C1',
 'C2',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D10',
 'D11',
 'D15',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'PCA_V1',
 'PCA_V2',
 'PCA_V3',
 'PCA_V4',
 'PCA_V5',
 'PCA_V6',
 'PCA_V7',
 'PCA_V8',
 'PCA_V9',
 'PCA_V10',
 'PCA_V11',
 'PCA_V12',
 'PCA_V13',
 'PCA_V14',
 'PCA_V15',
 'PCA_V16',
 'PCA_V17',
 'PCA_V18',
 'PCA_V19',
 'PCA_V20',
 'PCA_V21',
 'PCA_V22',
 'PCA_V23',
 'PCA_V24',
 'PCA_V25',
 'PCA_V26',
 'PCA_V27',
 'PCA_V28',
 'PCA_V29',
 'PCA_V30',
 'dist2',
 'C3',
 'id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_